In [2]:
import numpy as np
from scipy import linalg
from scipy.stats import norm

In [4]:
def solve_sym(xtx, b):
    return (linalg.lapack.dpotrs(linalg.lapack.dpotrf(xtx)[0], b)[0])

In [19]:
def turnbits_rec(p):
    if(p==1):
        return np.array([[False], [True]])
    else:
        tmp1 = np.c_[np.array([False]*pow(2, p-1)).reshape(-1, 1), turnbits_rec(p-1)]
        tmp2 = np.c_[np.array([True]*pow(2, p-1)).reshape(-1, 1), turnbits_rec(p-1)]
        return np.r_[tmp1, tmp2]

In [68]:
# Create data
n = 1000
p = 9
x = norm.rvs(size=(n, p))
x = np.c_[np.ones((n, 1)), x]
beta = np.arange(1, p+2)
# 用index的列创建y
index = [2, 3, 6]
y = x[:, index].dot(beta[index]).reshape(-1, 1) + norm.rvs(size = (n, 1))#这里数组间会进行广播
xtx = x.T.dot(x)#先建立全数据的xtx,xty，对于子集回归直接取其行和列，不用再单独运算和存储
xty = x.T.dot(y)
#创建子集
columns = turnbits_rec(p)
columns = np.c_[np.array([True]*columns.shape[0]).reshape(-1, 1), columns]
#解y的估计值
coef = [solve_sym(xtx[ind][:, ind], xty[ind]) for ind in columns]
#计算mse
def mse(x, y, b):
    return np.sum((y-x.dot(b))*(y-x.dot(b)))#sum(list(map(pow, y-x.dot(b), 2))), map后的每个元素必须是等长列表
x_mse = [x[:, ind] for ind in columns]
y_mse = [y for ind in columns]
result = list(map(mse, x_mse, y_mse, coef))#这里coef每个元素长度不想等，必须有对应的x与其匹配

In [78]:
result = np.array(result)

m = np.argmin(result)

columns[m]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

虽然是选用的index是[2, 3, 6],但全系数的拟合效果仍然比局部要好

原因可能在于误差项相对于自变量扰动较大

In [88]:
result

array([75505.99997236, 75290.23543947, 75499.48605662, 75283.87865356,
       75095.46301519, 74914.14673814, 75088.56039318, 74907.40876312,
       25371.96123184, 25371.6611847 , 25363.9150298 , 25363.60810454,
       25353.9731682 , 25353.34740632, 25345.83948939, 25345.20279833,
       75492.07256599, 75271.24473991, 75485.6958844 , 75265.0482384 ,
       75080.43135806, 74894.37709863, 75073.67496823, 74887.80710606,
       25371.90336165, 25371.59110306, 25363.84678839, 25363.52648003,
       25353.93125487, 25353.29020739, 25345.78867505, 25345.13510042,
       75425.62001478, 75221.2109081 , 75419.90089691, 75215.58002363,
       75014.63038161, 74843.80957816, 75008.5486665 , 74837.8258076 ,
       25340.71458819, 25340.6132705 , 25332.08146611, 25331.97740247,
       25323.00561121, 25322.69137075, 25314.2852437 , 25313.96547434,
       75408.63094766, 75199.08095307, 75403.0689305 , 75193.62914185,
       74996.41270523, 74820.80597916, 74990.49820318, 74815.01002215,
      